<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1


    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-04 13:20:30
-------------------
qualified stocks: 92
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.53 L
Current:  1.36 C
-------------------
Today PnL: 41.50 K (0.31%)
Current PnL: -18.78 L (-12.77%)
CY Booked + Current PnL: -10.51 L (-7.15%)
-------------------
Total profit:  2.44 L
Total loss:  -21.22 L
-------------------
Total Booked + Current PnL: 16.47 L (13.72%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.08%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.13 C
Est FTT PnL: 77.07 L (56.63%)
Deployed:  1.20 C
Current:  1.36 C
CAGR/XIRR %: 8.23%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,761.55,-20.73,H-LC,5.17,109776.0,5464.0,12075.0,2.03,5.24,11.00,16.81,10.0,0.45,0.83,22.13,X5K,ATH,METALS
27,DIXON,14367.00,18940.15,-26.18,H-MC,6.36,135840.0,20904.0,15676.0,0.78,18.19,11.54,31.83,85.0,1.33,1.02,52.88,X40N,ATH,IT
71,SIS,477.00,477.00,2283.94,H-SC,6.81,64141.0,-16949.0,16946.0,-0.75,-20.90,26.42,0.00,157.0,-1.00,0.48,29.99,OX40N,NTT,MISC
82,TTKPRESTIG,769.29,770.00,-9.79,H-SC,11.92,82537.0,-18240.0,18331.0,-0.28,-18.10,22.21,0.09,145.0,-1.00,0.62,8.43,OX40N,NTT,DURABLES
77,TATAELXSI,7332.28,7332.00,-19.66,H-MC,1.94,84287.0,-18365.0,18358.0,-0.56,-17.89,21.78,-0.00,101.0,-1.00,0.64,28.13,OX40N,NTT,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,WIPRO,243.46,420.0,-11.48,M-LC,5.2,151826.0,881.0,108571.0,0.88,0.58,71.51,72.51,55.0,0.01,1.15,6.64,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-21.14,H-LC,1.79,207719.0,3219.0,38200.0,0.89,1.57,18.39,20.25,9.0,0.08,1.57,10.44,X40N,BTT,PHARMA
53,LTIM,5564.16,7230.2,-5.37,H-LC,3.00,182250.0,-18060.0,78039.0,0.90,-9.02,42.82,29.94,24.0,-0.23,1.37,25.91,X200,ATH,IT
5,ASIANPAINT,2961.56,4250.0,-16.24,H-LC,3.02,166328.0,-35058.0,122667.0,0.62,-17.41,73.75,43.51,19.0,-0.29,1.25,15.78,X40,BTT,PAINTS
10,BAJAJHFL,122.26,152.0,-10.01,H-LC,3.17,186780.0,-13726.0,62497.0,0.87,-6.85,33.46,24.33,30.0,-0.22,1.41,7.71,X40N,BTT,FINANCE
57,NESTLEIND,2268.60,2755.0,-16.15,H-LC,4.40,266035.0,609.0,56293.0,-0.12,0.23,21.16,21.44,26.0,0.01,2.01,6.89,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SFL,1016.35,1287.00,37.16,M-SC,13.01,181000.0,-81218.0,151044.0,4.27,-30.97,83.45,26.63,231.0,-0.54,1.37,12.54,XY24,NTT,MISC
64,SAIL,130.64,228.00,79.57,M-MC,11.47,215663.0,-9299.0,176951.0,4.26,-4.13,82.05,74.53,196.0,-0.05,1.63,25.21,XY24,BTT,STEEL
87,VALIANTORG,512.64,838.00,-315.22,H-SC,20.49,91494.0,-41792.0,126390.0,4.16,-31.36,138.14,63.47,147.0,-0.33,0.69,53.37,XR,NTT,CHEMICALS
35,HEROMOTOCO,4311.81,5949.07,-2.86,H-MC,4.03,156065.0,5152.0,52157.0,3.42,3.41,33.42,37.97,90.0,0.10,1.18,29.58,AR,ATH,AUTO
56,NATIONALUM,189.63,247.44,-50.91,H-MC,2.99,98967.0,-1537.0,32174.0,3.34,-1.53,32.51,30.49,81.0,-0.05,0.75,30.38,MH,ATH,METALS


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,SYMPHONY,1306.42,1306.00,-35.83,M-SC,8.89,136751.0,-34390.0,34338.0,-4.39,-20.09,25.11,-0.03,197.0,-1.00,1.03,0.00,OX40N,NTT,DURABLES
39,HONAUT,42646.15,58357.33,-22.51,H-MC,2.37,111735.0,-16203.0,63343.0,-3.82,-12.67,56.69,36.84,106.0,-0.26,0.84,14.39,X40N,ATH,ELECTRICAL
42,INDIAMART,2327.09,4850.92,-48.64,H-SC,13.44,133830.0,10494.0,123271.0,-2.84,8.51,92.11,108.45,118.0,0.09,1.01,32.76,AR,ATH,MISC
3,ALKYLAMINE,2347.98,4546.37,5.08,H-SC,19.58,97795.0,-3168.0,97697.0,-2.81,-3.14,99.90,93.63,149.0,-0.03,0.74,45.66,SR,ATH,CHEMICALS
0,5PAISA,564.75,565.00,128.37,H-SC,5.41,123115.0,-63817.0,63897.0,-2.40,-34.14,51.90,0.04,153.0,-1.00,0.93,17.78,OX40N,NTT,FINANCE


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,INDIGOPNTS,1407.73,1408.0,126.00,H-SC,5.38,148329.0,-26230.0,26269.0,-0.70,-15.03,17.71,0.02,135.0,-1.00,1.12,28.62,OX40N,NTT,PAINTS
83,UJJIVANSFB,52.77,53.0,36.52,H-SC,20.77,118557.0,-23922.0,24541.0,1.60,-16.79,20.70,0.44,163.0,-0.97,0.89,39.93,OX40N,NTT,BANKS
20,CERA,7989.07,8422.0,-17.18,H-SC,2.73,118224.0,-25579.0,33375.0,2.08,-17.79,28.23,5.42,155.0,-0.77,0.89,30.46,OX40N,NTT,CERAMICS
49,KANSAINER,299.63,340.0,-68.20,H-SC,1.72,221616.0,-48051.0,84391.0,-1.52,-17.82,38.08,13.47,136.0,-0.57,1.67,12.09,XY24,NTT,PAINTS
77,TATAELXSI,7332.28,7332.0,-19.66,H-MC,1.94,84287.0,-18365.0,18358.0,-0.56,-17.89,21.78,-0.00,101.0,-1.00,0.64,28.13,OX40N,NTT,IT


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,3794.03,4998.00,-27.38,H-LC,14.25,237089.0,-58845.0,152756.0,1.22,-19.88,64.43,31.73,1.0,-0.39,1.79,1.22,X40,BTT,IT
88,VBL,492.64,671.64,-9.72,H-LC,11.65,273923.0,8883.0,87409.0,-0.54,3.35,31.91,36.33,3.0,0.10,2.07,17.10,X40N,ATH,FMCG
45,INFY,1461.46,2275.00,-17.80,H-LC,7.82,263291.0,1690.0,143941.0,0.09,0.65,54.67,55.67,4.0,0.01,1.99,6.79,X40,BTT,IT
1,ABB,5551.76,7934.00,-41.13,H-LC,7.66,210112.0,-6407.0,99320.0,-2.22,-2.96,47.27,42.91,5.0,-0.06,1.59,9.83,AR,NTT,ELECTRICAL
34,HCLTECH,1562.67,1943.91,0.97,H-LC,8.11,178779.0,-11867.0,58371.0,0.92,-6.22,32.65,24.40,8.0,-0.20,1.35,8.72,X40,ATH,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,20.01,H-SC,12.98,153909.0,27709.0,18469.0,-0.8,21.96,12.0,36.59,156.0,1.5,1.16,37.22,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.64,31.89,H-SC,13.09,134770.0,9473.0,49609.0,2.10,7.56,36.81,47.15,144.0,0.19,1.02,68.61,MH,ATH,POWER
55,MEDANTA,1087.93,1486.00,20.01,H-SC,12.98,153909.0,27709.0,18469.0,-0.80,21.96,12.00,36.59,156.0,1.50,1.16,37.22,XY24,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-2.86,H-MC,4.03,156065.0,5152.0,52157.0,3.42,3.41,33.42,37.97,90.0,0.10,1.18,29.58,AR,ATH,AUTO
25,DABUR,505.20,735.00,-4.13,H-MC,5.16,207772.0,10744.0,78870.0,-0.30,5.45,37.96,45.49,100.0,0.14,1.57,19.20,XY24,BTT,FMCG
19,CAMS,3643.00,5226.82,-6.36,H-SC,1.20,105104.0,3100.0,41243.0,3.00,3.04,39.24,43.48,120.0,0.08,0.79,20.33,X40N,ATH,MISC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.64,31.89,H-SC,13.09,134770.0,9473.0,49609.0,2.10,7.56,36.81,47.15,144.0,0.19,1.02,68.61,MH,ATH,POWER
90,WHIRLPOOL,1219.98,2270.00,-40.32,M-SC,6.65,100500.0,9002.0,69747.0,1.85,9.84,69.40,86.07,211.0,0.13,0.76,45.38,XR,NTT,DURABLES
35,HEROMOTOCO,4311.81,5949.07,-2.86,H-MC,4.03,156065.0,5152.0,52157.0,3.42,3.41,33.42,37.97,90.0,0.10,1.18,29.58,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-48.64,H-SC,13.44,133830.0,10494.0,123271.0,-2.84,8.51,92.11,108.45,118.0,0.09,1.01,32.76,AR,ATH,MISC
91,WIPRO,243.46,420.00,-11.48,M-LC,5.20,151826.0,881.0,108571.0,0.88,0.58,71.51,72.51,55.0,0.01,1.15,6.64,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,SIS,477.00,477.00,2283.94,H-SC,6.81,64141.0,-16949.0,16946.0,-0.75,-20.90,26.42,0.00,157.0,-1.00,0.48,29.99,OX40N,NTT,MISC
23,COFFEEDAY,59.14,80.00,-53.72,L-SC,21.28,67469.0,-46080.0,86131.0,-0.51,-40.58,127.66,35.27,267.0,-0.53,0.51,65.13,XR,NTT,HOTELS
54,MASFIN,326.60,399.50,-19.60,H-SC,16.64,91320.0,-6660.0,28528.0,-0.20,-6.80,31.24,22.32,151.0,-0.23,0.69,32.45,XR,ATH,FINANCE
3,ALKYLAMINE,2347.98,4546.37,5.08,H-SC,19.58,97795.0,-3168.0,97697.0,-2.81,-3.14,99.90,93.63,149.0,-0.03,0.74,45.66,SR,ATH,CHEMICALS
36,HINDALCO,651.95,761.55,-20.73,H-LC,5.17,109776.0,5464.0,12075.0,2.03,5.24,11.00,16.81,10.0,0.45,0.83,22.13,X5K,ATH,METALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,20.01,H-SC,12.98,153909.0,27709.0,18469.0,-0.80,21.96,12.00,36.59,156.0,1.50,1.16,37.22,XY24,NTT,HEALTHCARE
27,DIXON,14367.00,18940.15,-26.18,H-MC,6.36,135840.0,20904.0,15676.0,0.78,18.19,11.54,31.83,85.0,1.33,1.02,52.88,X40N,ATH,IT
88,VBL,492.64,671.64,-9.72,H-LC,11.65,273923.0,8883.0,87409.0,-0.54,3.35,31.91,36.33,3.0,0.10,2.07,17.10,X40N,ATH,FMCG
69,SHALBY,261.39,327.00,957.42,M-SC,8.96,143867.0,-37276.0,82738.0,0.92,-20.58,57.51,25.10,233.0,-0.45,1.09,15.29,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-25.21,H-SC,6.06,212928.0,19908.0,28128.0,-1.44,10.31,13.21,24.89,128.0,0.71,1.61,35.81,X40N,NTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.74
1,25,41.73
2,50,71.44


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.70
LC    37.17
MC    22.20
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.12
X40N     15.13
X40      13.23
XY25     11.76
XR        8.98
OX40N     7.71
AR        7.61
X5K       2.21
MH        1.77
X200      1.37
SR        1.18
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    29.12
H-SC    27.26
H-MC    19.24
M-SC    12.02
M-LC     7.01
M-MC     2.62
L-SC     1.42
L-LC     1.04
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.38,-1.20,33.74
IT,11.94,-17.95,75.25
FINANCE,8.37,-17.67,60.37
BANKS,7.44,-15.59,62.75
PAINTS,5.79,-13.65,36.05
MISC,5.14,-19.83,89.93
HEALTHCARE,5.08,2.27,22.40
ELECTRICAL,4.75,-11.49,59.77
AUTO,4.35,-17.31,68.35


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2782592.0,23
XR,1103178.0,13
AR,1057028.0,9
X40,819127.0,10
X40N,603642.0,12
XY25,493845.0,8
OX40N,354134.0,10
SR,187601.0,2
X5K,146464.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2710441.0,30
M-SC,1379017.0,15
H-LC,1297792.0,20
H-MC,1157612.0,15
M-LC,450000.0,5
M-MC,331354.0,2
L-SC,272897.0,3
L-MC,57112.0,1
L-LC,51208.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      929402.0      7
M-SC       XY24      844028.0      7
H-SC       AR        623924.0      4
H-LC       X40       546354.0      6
H-SC       XR        513778.0      6
H-MC       XY24      451614.0      4
M-MC       XY24      331354.0      2
H-LC       X40N      301855.0      6
           AR        224063.0      2
H-SC       X40N      222768.0      4
H-MC       XY25      205929.0      2
           X40       197090.0      3
M-LC       XY24      187965.0      2
H-SC       SR        187601.0      2
L-SC       XR        184731.0      2
H-SC       OX40N     183359.0      6
M-SC       AR        156884.0      2
M-LC       X5K       134389.0      1
H-MC       XR        121271.0      1
M-SC       XY25      120456.0      1
           XR        117715.0      2
M-LC       XR        108571.0      1
H-LC       XY25       97177.0      3
L-SC       OX40N      88166.0      1
H-MC       X40N       79019.0      2
H-LC       X200       78039.0      1
M-SC       X40        75683.0      1
           OX40N      64251.0      2
L-MC       XR         57112.0      1
H-MC       AR         52157.0      1
L-LC       XY25       51208.0      1
H-SC       MH         49609.0      1
H-LC       XY24       38229.0      1
H-MC       MH         32174.0      1
M-LC       XY25       19075.0      1
H-MC       OX40N      18358.0      1
H-LC       X5K        12075.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
